In [1]:
import requests
import pandas as pd

In [2]:
# Get IUCN API key
from api_keys import iucn_key

In [3]:
# URL to get Australian species from the IUCN API 
url1 = "http://apiv3.iucnredlist.org/api/v3/country/getspecies/AU?token="
# To include the API key
query_url = f"{iucn_key}"

In [4]:
# The total url, return in json format
iucn_species_australia = requests.get(url1 + query_url).json()

In [5]:
# Retain the results contained within 'result'
iucn_au_results = iucn_species_australia["result"]

In [6]:
# Create a dataframe for the results
iucn_au_df = pd.DataFrame(iucn_au_results)

In [7]:
# Retain only the species that have 'endangered' status
endangered_au_species = iucn_au_df.category.str.contains('EN')

In [8]:
critical_endanger_au_species = iucn_au_df.category.str.contains('CR')

In [9]:
# Reduce the dataframe to include only endangered species from Australia
only_endangered = iucn_au_df[endangered_au_species]

In [10]:
only_critical_endanger = iucn_au_df[critical_endanger_au_species]

In [11]:
only_endangered = only_endangered[['scientific_name']]

In [12]:
only_critical_endanger = only_critical_endanger[['scientific_name']]

In [13]:
critical_and_endangered = pd.concat([only_endangered, only_critical_endanger], axis=0)
critical_and_endangered

,scientific_name
24,Acacia aprica
36,Acacia bifaria
37,Acacia brachypoda
42,Acacia chrysotricha
74,Acacia imitans
...,...
8796,Varanus mitchelli
8866,Wollemia nobilis
8905,Zaglossus bruijnii
8951,Zyzomys palatalis


In [15]:
critical_and_endangered.replace(' ', '%20',regex=True,inplace=True)
critical_and_endangered.replace("'", "", regex=True,inplace=True)

In [16]:
critical_and_endangered["common_name"] = ""

In [17]:
critical_and_endangered.reset_index(inplace = True, drop = True)

In [19]:
url2 = "http://apiv3.iucnredlist.org/api/v3/species/common_names/"
token = f"?token={iucn_key}"

In [20]:
for index, row in critical_and_endangered.iterrows():
# initial outdata
    outdata=""
#     get scientific name
    scientific_name = row['scientific_name']
#     sending request getting response for taxonname
    response = requests.get(url2 + scientific_name + token).json()
    
    if response["result"] == []:
        outdata = "NaN"
        
#     get dataline
        
    else:
        
        for i in response["result"]:
            dataline=response["result"][0]
    
            if dataline["primary"] == True:
                outdata=dataline["taxonname"]
                print(outdata)
            
            else:
                outdata=dataline["taxonname"]
                
    critical_and_endangered.loc[index, "common_name"] = outdata
 

Blunt Wattle
Western Wheatbelt Wattle
Newry Golden Wattle
Newry Golden Wattle
Gibson Wattle
Phantom Wattle
Dwarf Rock Wattle
Thylacine Darner
McCarthy’s Plant-louse
Waterwheel
Waterwheel
Waterwheel
Waterwheel
Pilbara Dragon
Gnaraloo Worm-lizard
Noisy Scrub-bird
Noisy Scrub-bird
Noisy Scrub-bird
Noisy Scrub-bird
Rufous Scrub-bird
Rufous Scrub-bird
Wallum Darner
Carnarvon Darner
Cape Leeuwin Snail
Cape Leeuwin Snail
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Hog Deer
Sei Whale
Sei Whale
Sei Whale
Sei Whale
Sei Whale
Sei Whale
Blue Whale
Blue Whale
Blue Whale
Northern Bettong
Australasian Bittern
Great Knot
Great Knot
Pig-nosed Turtle
Pig-nosed Turtle
Pig-nosed Turtle
Pig-nosed Turtle
Pig-nosed Turtle
Green Turtle
Green Turtle
Green Turtle
Green Turtle
Green Turtle
Green Turtle
Kurnbudj Ctenotus
Kurnbudj Ctenotus
Stuart's Ctenotus
Stuart's Ctenotus
Alpine She-Oak Slender Bluetongue
Alpine She-Oak Slender Bluetongue
Coxen's Fig-parrot
Eastern Bristlebird
Northe

In [21]:
critical_and_endangered

,scientific_name,common_name
0,Acacia%20aprica,Blunt Wattle
1,Acacia%20bifaria,NaN
2,Acacia%20brachypoda,Western Wheatbelt Wattle
3,Acacia%20chrysotricha,Newry Golden Wattle
4,Acacia%20imitans,Gibson Wattle
...,...,...
364,Varanus%20mitchelli,Mitchell's Water Monitor
365,Wollemia%20nobilis,Wollemi Pine
366,Zaglossus%20bruijnii,Western Long-beaked Echidna
367,Zyzomys%20palatalis,Carpentarian Rock-rat


In [22]:
critical_and_endangered.to_csv("au_critical_and_endanger_MOSHER.csv", index=False)